In [1]:
%matplotlib widget

import os
import numpy as np
import pandas as pd
from copy import deepcopy as copy
from astropy.table import Table

from lvmdrp import path, __version__ as drpver
from lvmdrp.core.constants import ARC_LAMPS
from lvmdrp.utils import metadata as md
from lvmdrp.functions import run_drp as drp

from lvmdrp.functions import imageMethod, rssMethod


import matplotlib.pyplot as plt
from lvmdrp.core.spectrum1d import Spectrum1D


def subtract_continuum(in_rss, out_rss, median_box=30, thresh=1.2, niter=5, mask_delt=1):
    arc = rssMethod.loadRSS(in_rss)
    
    cont_rss = np.zeros_like(arc._data)
    for ifiber in range(arc._fibers):
        if ifiber not in arc._good_fibers:
            continue
        spec = arc.getSpec(ifiber)

        spec_s = copy(spec)
        spec_s.smoothSpec(size=median_box, method="median")

        cur_mask = 0
        for i in range(niter):
            mask = np.divide(spec._data, spec_s._data, where=spec_s._data != 0, out=np.zeros_like(spec_s._data)) < thresh

            spec_s = Spectrum1D(data=np.interp(spec._pixels, spec._pixels[mask], spec._data[mask]))
            spec_s.smoothSpec(size=30, method="median")

            if np.abs(cur_mask - mask.sum()) <= mask_delt: break
            cur_mask = mask.sum()

        # define continuum with last iteration's mask
        mask = np.divide(spec._data, spec_s._data, where=spec_s._data != 0, out=np.zeros_like(spec_s._data)) < thresh
        cont_rss[ifiber] = np.interp(spec._pixels, spec._pixels[mask], spec._data[mask])
    
    out_con = copy(arc)
    out_con.setData(data=cont_rss)
    new_arc = copy(arc)
    new_arc.setData(data=arc._data - cont_rss)
    
    new_arc.writeFitsData(out_rss)
    
    return new_arc, out_con


ORIG_MASTER_DIR = os.getenv("LVM_MASTER_DIR")
MASTER_CON_LAMPS = {"b": "ldls", "r": "ldls", "z": "quartz"}
MASTER_ARC_LAMPS = {"b": "hgne", "r": "neon", "z": "neon"}
SLITMAP = Table(drp.fibermap.data)

In [2]:
frames_table = md.get_metadata(tileid="*", mjd=60177)
frames_table.query("imagetyp == 'bias' | imagetyp == 'dark' | imagetyp == 'flat' | imagetyp == 'arc'", inplace=True)
frames_table = frames_table.loc[frames_table.name.str.contains("sdR")]

[INFO]: loading metadata store at /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5
[INFO]: found 513 frames in store '/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/raw_metadata.hdf5'
[INFO]: number of frames after filtering 513
[INFO]: total number of frames found 513


In [3]:
masters_mjd = frames_table.mjd.min()
masters_path = os.path.join(ORIG_MASTER_DIR, f"{masters_mjd}")
os.makedirs(masters_path, exist_ok=True)

In [4]:
lamp_combs = [("hgne", "argon", "xenon", "neon")]
arc_analogs = frames_table.query("imagetyp=='arc'").groupby(["camera"])

tileid = frames_table.tileid.iloc[0]

for combs in lamp_combs:    
    for camera in arc_analogs.groups:

        # define input/output paths
        warc_i_path = ",".join([path.full("lvm_master", drpver=drpver, tileid=tileid, mjd=masters_mjd, kind=f"wmarc_{lamp}", camera=camera) for lamp in combs])
        warc_c_path = path.full("lvm_master", drpver=drpver, tileid=tileid, mjd=masters_mjd, kind=f"wmarc_{'_'.join(combs)}", camera=camera)
        
        # combine RSS files
        rssMethod.combineRSS_drp(in_rsss=warc_i_path, out_rss=warc_c_path, method="sum")
        
        # subtract continuum from each fiber
        subtract_continuum(in_rss=warc_c_path, out_rss=warc_c_path, median_box=50, thresh=1.001, niter=1)
        
        # fix metadata and additional extensions
        rss_c = rssMethod.loadRSS(warc_c_path)
        rss_c.setSlitmap(Table(drp.fibermap.data))
        for lamp in combs:
            rss_c._header[lamp.upper()] = "ON"
        rss_c._header["OBJECT"] = ", ".join([lamp.capitalize() for lamp in combs])
        rss_c.writeFitsData(warc_c_path)
        print("combined lamps: ", warc_c_path)

combined lamps:  /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-b1.fits
combined lamps:  /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-b2.fits
combined lamps:  /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-b3.fits
combined lamps:  /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-r1.fits
combined lamps:  /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-r2.fits
combined lamps:  /home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-r3.fits
combined lamps: 

In [5]:
# in_rss = "/home/mejia/Research/lvm/lvmdata/data.sdss5.org/sas/sdsswork/lvm/spectro/redux/0.1.1.dev0/1111/60177/calib/lvm-wmarc_hgne_argon_xenon_neon-b1.fits"
# new_arc, new_con = subtract_continuum(in_rss=in_rss, out_rss="test_cont.fits", median_box=50, thresh=1.001, niter=1)
# print(new_con._data.mean())